In [8]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
import os
from dotenv import load_dotenv

embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-large"
)
# Specify the folder and file
api_keys_env_file = "../api_keys.env"

# Load the .env file
load_dotenv(api_keys_env_file)

# Access the variables
openai_api_key = os.getenv("OPENAI_API_KEY")

# Setup vector DB
chroma_client = chromadb.PersistentClient(
    path="../vector_db",
)
pdf_collection = chroma_client.get_or_create_collection("pdf_data")
html_collection = chroma_client.get_or_create_collection("html_data")
pdf_vector_store = ChromaVectorStore(chroma_collection=pdf_collection)
html_vector_store = ChromaVectorStore(chroma_collection=html_collection)
pdf_index = VectorStoreIndex.from_vector_store(
    pdf_vector_store,
    embed_model=embed_model,
)
html_index = VectorStoreIndex.from_vector_store(
    html_vector_store,
    embed_model=embed_model,
)

In [20]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# query_engine_prompt = PromptTemplate("""
# Based on the provided context, answer the posed question in Hungarian language.\n
# If the answer to the question is not clear based on the context, respond with: No data found relevant to the user's question.\n
# Context: {context_str}\n
# Question: {question_str}\n
# Answer: """)

pdf_engine = pdf_index.as_query_engine(llm=None, similarity_top_k=3)
html_engine = html_index.as_query_engine(llm=None, similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=pdf_engine,
        metadata=ToolMetadata(
            name="BME_Rulebook_Data_Store",
            description=(
                "This tool is a data store, containing chunks of legal regulations and rules that specify how BME, a Hungarian University functions."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=html_engine,
        metadata=ToolMetadata(
            name="BME_FAQ_Website_Data_Store",
            description=(
                "This tool is a data store, containing frequently asked questions and their answers from the website of the Central Study Office of BME, a Hungarian university. The data contains information about the use of university systems, academic matters, finances, student ID card creation and student loans."
            ),
        ),
    ),
]

In [21]:
from llama_index.core.memory import ChatMemoryBuffer

system_prompt = """Personality:
You are an AI agent of the Hungarian University called BME. Your task is to query the data tools that are available to you and provide an accurate answer based on only what your data sources return.
Instructions:
- You have two tools. One contains answers to frequently asked questions often asked by the students of the university and the other contains detailed information about the rules and laws the university operates by.
- Always use the information from the available tools.
- If the data tools do not provide any information relevant to the user's question, politely inform the user that you don't know the answer to their question.
- You must respond in Hungarian language."""

llm = OpenAI(model="gpt-4o-mini", system_prompt=system_prompt, api_key=openai_api_key)

memory = ChatMemoryBuffer(token_limit=8000)

agent = OpenAIAgent(
    tools=query_engine_tools,
    llm=llm,
    memory=memory,
    prefix_messages=[],
    verbose=True
)

In [24]:
response = agent.chat("Milyen bizottságok működnek állandóan az egyetemen belül?")

Added user message to memory: Milyen bizottságok működnek állandóan az egyetemen belül?
=== Calling Function ===
Calling function: BME_Rulebook_Data_Store with args: {"input":"állandó bizottságok az egyetemen"}
Got output: Egyetemi Tanulmányi Bizottság (ETB), Egyetemi Szociális Bizottság (ESzB), Kari Tanulmányi Bizottság (KTB), valamint Egyetemi Kreditátviteli Bizottság (EKB) az állandó bizottságok az egyetemen.



In [25]:
print(response)

Az egyetemen belül az alábbi állandó bizottságok működnek:

1. Egyetemi Tanulmányi Bizottság (ETB)
2. Egyetemi Szociális Bizottság (ESzB)
3. Kari Tanulmányi Bizottság (KTB)
4. Egyetemi Kreditátviteli Bizottság (EKB)
